In [9]:
import numpy as np
from utils.general import info,ok, warning
import os

GM1 = '/home/ec2-user/SageMaker/data/GM_all_1945_1956/'
GM2 = '/home/ec2-user/SageMaker/data/GM_all_1957-1967/'

files = [GM1+f for f in os.listdir(GM1)]+[GM2+f for f in os.listdir(GM2)]
info(f'Cantidad de archivos {len(files):,}')

rndg = np.random.default_rng(2022)
# print(files[:10])
rndg.shuffle(files)
# print(files[:10])

files = files[:1000000]
info(f'Cantidad de archivos {len(files):,}')


2022-03-26 21:00:54.066413 [ INFO  ] Cantidad de archivos 2,057,868
2022-03-26 21:00:54.685888 [ INFO  ] Cantidad de archivos 1,000,000


In [8]:

# warning('Computing labeled data')
# #########################
# # NOT MENTIONING CANADA #
# #########################
# IN_CANADA_FILE = '../05. High Recall V2/auxiliary_notebooks/in_canada.csv'
# assert os.path.isfile(IN_CANADA_FILE)

# lines = open(IN_CANADA_FILE,'r').read().splitlines()
# ids_set = set([line.split(';')[0] for line in lines if line.endswith('False')])
# # labeled_data = [DataItem(id_) for id_ in irrelevant_ids]


# #############################
# # LABELED OVER THREE ROUNDS #
# #############################
# for line in open('../05. High Recall V2/labeled_data.csv').read().splitlines()[1:]:
#     id_,label = line.split(';')
#     ids_set.add(id_)
# len(ids_set)

2022-03-24 14:54:05.759820 [WARNING] Computing labeled data


114492

In [10]:
# from utils.general import id2file
# files = [id2file(id_) for id_ in ids_set]
# files[:10]

['/home/ec2-user/SageMaker/data/GM_all_1957-1967/1288562627.xml',
 '/home/ec2-user/SageMaker/data/GM_all_1957-1967/1288387755.xml',
 '/home/ec2-user/SageMaker/data/GM_all_1957-1967/1323532462.xml',
 '/home/ec2-user/SageMaker/data/GM_all_1945_1956/1289625759.xml',
 '/home/ec2-user/SageMaker/data/GM_all_1945_1956/1287366275.xml',
 '/home/ec2-user/SageMaker/data/GM_all_1945_1956/1289454772.xml',
 '/home/ec2-user/SageMaker/data/GM_all_1957-1967/1282877585.xml',
 '/home/ec2-user/SageMaker/data/GM_all_1957-1967/1270333241.xml',
 '/home/ec2-user/SageMaker/data/GM_all_1945_1956/1291769709.xml',
 '/home/ec2-user/SageMaker/data/GM_all_1945_1956/1323603473.xml']

In [10]:
%%time
from collections import defaultdict
import spacy
from utils.tdmstudio import TDMStudio
import string
from tqdm import tqdm

from utils.models import tokenize
# AUXs
# nlp = spacy.load('en_core_web_sm', disable=['textcat', 'parser','ner'])

# def remove_punctuation(word):
#     return ''.join([char for char in word if not char in string.punctuation+' '])

# def tokenize(str_):
#     tokens = [word.lemma_.lower() for word in nlp(str_) if not word.is_stop and word.lemma_.isalnum()]
#     tokens = [word.replace('\n', '') for word in tokens if not word.isnumeric() and len(remove_punctuation(word.replace('\n', '')))!=0]
#     return tokens


from threading import Lock
import gc

def process_file(idx, file_):
    title, text = TDMStudio.get_title_and_text(file_)
    id_ = file_.split('/')[-1][:-4]
    # Write
#     wlock.acquire()
    writer0.write(f'{id_}\n')
    writer0.flush()
#     wlock.release()
    
    if not title is None and not text is None:        
        tokens = tokenize(f'{title}. {text}') #tokenize(title) + tokenize(text)
    
        tokens_set = set(tokens)

        ############################################
        # UPDATE freq (remove low frequency terms) #
        ############################################
        if (idx+1)%25000==0:
            to_remove = [word for word in freq if freq[word]<25]
            info(f'Ite. no. {idx:10,} - No. of words to remove {len(to_remove):12,} - Remaining {len(freq)-len(to_remove)}')
            for word in to_remove:
                del(freq[word])
            while len(to_remove)>0:
                del(to_remove[0])
            del(to_remove)
            gc.collect()
        for token in tokens_set:
            freq[token]+=1

        del(tokens_set,tokens)
    del(title,text, id_)
    



CPU times: user 3.98 s, sys: 721 ms, total: 4.7 s
Wall time: 9.92 s


In [1]:
!wc -l done_vocab.txt

1586746 done_vocab.txt


In [11]:
len(files)

1000000

In [ ]:
%%time
from utils.general import info, ok
import concurrent.futures
import datetime
from tqdm import tqdm

writer0 = open('done_vocab.txt', 'w')

info('Starting...')
freq = defaultdict(int)

for idx, file_ in tqdm(enumerate(files)):
    process_file(idx,file_)
# with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
#     executor.map(process_file, files, chunksize=4000)
    
info('Saving freq.txt')
writer = open('precomputed/freq.txt', 'w')
for word in freq:
    if freq[word]>205:
        writer.write(f'{word};{freq[word]}\n')
writer.close()

# info('Saving vocab.txt')
# writer = open('precomputed/vocab.txt', 'w')
# for word in freq:
#     if freq[word]>205:
#         writer.write(f'{word};{freq[word]}\n')
# writer.close()


writer0.close()
ok('Done!')

2022-03-26 21:01:32.917810 [ INFO  ] Starting...


24995it [09:38, 37.98it/s]

2022-03-26 21:11:12.208105 [ INFO  ] Ite. no.     24,999 - No. of words to remove      711,425 - Remaining 22758


49997it [20:28, 60.33it/s]

2022-03-26 21:22:01.356605 [ INFO  ] Ite. no.     49,999 - No. of words to remove      730,758 - Remaining 24914


73123it [30:34, 43.09it/s]

In [2]:
!date


Thu Mar 24 11:57:11 UTC 2022


In [4]:
!ls -lh precomputed/freq.txt
!ls -lh done_vocab.txt

-rw-rw-r-- 1 ec2-user ec2-user 14K Mar 24 21:43 precomputed/freq.txt
-rw-rw-r-- 1 ec2-user ec2-user 108K Mar 24 21:43 done_vocab.txt


In [5]:
!wc -l precomputed/freq.txt
!wc -l done_vocab.txt

1441 precomputed/freq.txt
10000 done_vocab.txt


In [ ]:
threshold = int(len(files)*(0.01/100.0))
info(f'threshold={threshold}')
vocab = [word for word in freq if freq[word]>threshold]
info(f'len(vocab)={len(vocab)}')

In [15]:
assert os.path.isfile(files[-1])

In [17]:
from utils.tdmstudio import TDMStudio
from tqdm import tqdm
corpus=[]
for file_ in tqdm(files):
    title, text = TDMStudio.get_title_and_text(file_)
    corpus.append(f'{title}. {text}')
    del(title,text)
print(len(corpus))

100%|██████████| 2057868/2057868 [57:24<00:00, 597.38it/s]  

2057868


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from utils.models import tokenize
from utils.tdmstudio import TDMStudio
import spacy
import string

# AUXs
nlp = spacy.load('en_core_web_sm', disable=['textcat', 'parser','ner'])

def remove_punctuation(word):
    return ''.join([char for char in word if not char in string.punctuation+' '])

def tokenize(str_):
    tokens = [word.lemma_.lower() for word in nlp(str_) if not word.is_stop]
    tokens = [word.replace('\n', '') for word in tokens if not word.isnumeric() and len(remove_punctuation(word))!=0]
    return tokens

# STOPWORDS
stopwords = nlp.Defaults.stop_words
invalid = set([sw for sw in stopwords if any([token for token in tokenize(sw) if not token in stopwords ])]) # ['‘ve', "'m", '’ve', "'ve", '’m', '‘m', '‘d', '‘ll']
stopwords = set(stopwords.difference(invalid))

#MODEL
vectorizer = TfidfVectorizer(
                             input='content',
                             lowercase=True,
                             preprocessor=None,
                             tokenizer=tokenize,
                             analyzer='word',
                             stop_words=list(stopwords),
#                              token_pattern=r"(?u)\b\w\w+\b", #selects tokens of 2 or more alphanumeric char (punctuation is completely ignored and treated as token separator)
#                                                              # UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None
    
                             ngram_range=(1,1), #lower and upper boundary of the range of n-values for different n-grams.
                             max_df=1.0, #ignore terms that have a document frequency strictly higher than given threshold
                             min_df=0.001, #ignore terms that have a document frequency strictly lower than given threshold
                             max_features=50000, #build a vocabulary that only considers the top max_features ordered by term frequency acoss the corpus
                             vocabulary=None, #vocabulary is determined from the input documents
                             norm='l2',
                             use_idf=False,
                             )
X = vectorizer.fit_transform(corpus)

In [7]:
len(gm.union(dp))

2057868

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from utils.models import tokenize
from utils.tdmstudio import TDMStudio
import spacy
import string

# AUXs
nlp = spacy.load('en_core_web_sm', disable=['textcat', 'parser','ner'])

def remove_punctuation(word):
    return ''.join([char for char in word if not char in string.punctuation+' '])

def tokenize(str_):
    tokens = [word.lemma_.lower() for word in nlp(str_) if not word.is_stop]
    tokens = [word.replace('\n', '') for word in tokens if not word.isnumeric() and len(remove_punctuation(word))!=0]
    return tokens

# STOPWORDS
stopwords = nlp.Defaults.stop_words
invalid = set([sw for sw in stopwords if any([token for token in tokenize(sw) if not token in stopwords ])]) # ['‘ve', "'m", '’ve', "'ve", '’m', '‘m', '‘d', '‘ll']
stopwords = set(stopwords.difference(invalid))



X = vectorizer.fit_transform(corpus)